First, let's import `mujoco` and some other useful libraries.

In [1]:
import mujoco as mj
from mujoco.glfw import glfw
import numpy as np
from callbacks import *
from scipy.spatial.transform import Rotation as R

We will now set our model path, and ask MuJoCo to setup the following:

* MuJoCo's `mjModel` contains the _model description_, i.e., all quantities that *do not change over time*. 
* `mjData` contains the state and the quantities that depend on it. In order to make an `mjData`, we need an `mjModel`. `mjData` also contains useful functions of the state, for e.g., the Cartesian positions of objects in the world frame.
* `mjvCamera` and `mjvOption` are for visualization. We don't have to worry about this for now.

In [2]:
xml_path = 'models/mushr_corridor.xml'
view = "third"
assert view in ["first","third"]
simend = 600

# MuJoCo data structures
model = mj.MjModel.from_xml_path(xml_path)  # MuJoCo model
data  = mj.MjData(model)                    # MuJoCo data
cam   = mj.MjvCamera()                        # Abstract camera
opt   = mj.MjvOption()                        # visualization options

Next, we set up the visualization code. You don't have to change any of this code for Assignment 0.

In [3]:
# Init GLFW, create window, make OpenGL context current, request v-sync
glfw.init()
window = glfw.create_window(800, 600, "Demo", None, None)
glfw.make_context_current(window)
glfw.swap_interval(1)

# initialize visualization data structures
mj.mjv_defaultCamera(cam)
mj.mjv_defaultOption(opt)
scene = mj.MjvScene(model, maxgeom=10000)
context = mj.MjrContext(model, mj.mjtFontScale.mjFONTSCALE_150.value)

cb = Callbacks(model,data,cam,scene)

# install GLFW mouse and keyboard callbacks
glfw.set_key_callback(window, cb.keyboard)
glfw.set_cursor_pos_callback(window, cb.mouse_move)
glfw.set_mouse_button_callback(window, cb.mouse_button)
glfw.set_scroll_callback(window, cb.scroll)

# Example on how to set camera configuration
cam.azimuth = -90 ; cam.elevation = -45 ; cam.distance =  13
cam.lookat =np.array([ 0.0 , 0.0 , 0.0 ])

Now let's do something with our differential drive car. Our car has two actuators that control the velocity of the wheels. We can directly command these actuators to achieve a particular velocity by accessing `data.ctrl` variable. For a simple controller, like the one we'll be using in this notebook, this is fine. But for a more complicated controller, this will make our main simulation loop very clunky.

Thankfully, MuJoCo lets us use a control _callback_ using the `set_mjcb_control` method. A callback function is a function passed into another function as an argument, which is then invoked inside the outer function to complete some kind of routine or action.

This way, we can define our controller outside the main simulation loop, and then MuJoCo will call it automatically!

In [4]:
class Controller:
    def __init__(self,model,data):
        # Initialize the controller here.
        pass
    
    def controller(self,model,data):
        data.ctrl[0] = 0
        data.ctrl[1] = 0
    
    def controller(self, model, data, steering, velocity):
        data.ctrl[0] = steering
        data.ctrl[1] = velocity

c = Controller(model,data)
# print(data.ctrl[0])
mj.set_mjcb_control(c.controller)

The below while loop will continue executing for `simend` seconds, where `simend` is the end time we defined above. MuJoCo lets us keep track of the total elapsed time using the `data.time` variable.

At a frequency of ~60Hz, it will step forward the simulation using the `mj_step` function. A more detailed explanation of what happens when you call `mj_step` is given [here](https://mujoco.readthedocs.io/en/latest/computation.html?highlight=mj_step#forward-dynamics). But for the sake of simplicity, you can asume that it applies the controls to the actuator, calculates the resulting forces, and computes the result of the dynamics.  

In [5]:
def check_collision():
    # print("data.site_xpos: ", data.site_xpos)
    buddy_pos = data.site_xpos[0]
    human_pos = data.site_xpos[1]
    
    distance = (np.linalg.norm(buddy_pos - human_pos))
    # print(distance)
    #Explanation of 0.65224 given below
    # print("buddy_pos: ", buddy_pos)

    if distance <= 0.5:
        # print("distance <= 0.5")
        return True
    elif buddy_pos[0] <= -5.9 or buddy_pos[0] >= 5.9:
        # print("buddy_pos[0] <= -5.9 or buddy_pos[0] >= 5.9")
        return True
    elif buddy_pos[1] <= -5.9 or buddy_pos[1] >= 5.9:
        # print("buddy_pos[1] <= -5.9 or buddy_pos[1] >= 5.9")
        return True
    elif (buddy_pos[0] <= 3.7 and buddy_pos[0] >= -3.7) and (buddy_pos[1] <= -0.4 or buddy_pos[1] >= 0.4):
        # print("buddy_pos[0] <= 3.7 and buddy_pos[0] >= -3.7) and (buddy_pos[1] <= -0.425 or buddy_pos[1] >= 0.425")
        return True
        
    return False
    

In [6]:
prev_pos = [0, 0, 0]
checkpoints = [[5.0, 0, 0.5], [-5.0, 0, 0.5], [-5.0, -5.0, 0.5]]

def find_steering_angle(curr_vector, goal_vector):
    if not np.linalg.norm(curr_vector):
        return 0
    unit_vector_1 = curr_vector / np.linalg.norm(curr_vector)
    unit_vector_2 = goal_vector / np.linalg.norm(goal_vector)
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    angle = np.arccos(dot_product)

    cross = np.cross(goal_vector, curr_vector)
    if cross[2] > 0:
        angle = np.negative(angle)
  
    return angle


def find_goal_direction(goal_pos, curr_pos):
    if np.linalg.norm(goal_pos - curr_pos) == 0:
        return (goal_pos - curr_pos)
    return (goal_pos - curr_pos)/np.linalg.norm(goal_pos - curr_pos)


def check_proximity(curr_pos):
    global checkpoints
    goal_pos = checkpoints[0]
    #check distance
    dist = np.linalg.norm(curr_pos - goal_pos)
    if dist <= 0.75:
        #update checkpoints
        checkpoints.pop(0)


def move_buddy(curr_pos, agents, radius):
    global prev_pos, checkpoints
    
    #check proximity to goal
    check_proximity(curr_pos)
    
    if( len(checkpoints)==0):
            return
            
    goal_pos = checkpoints[0]
    # print("goal_pos: ", goal_pos)
    robot_vel = find_goal_direction(curr_pos, prev_pos)
    
    new_vel = RVO(curr_pos, robot_vel, goal_pos, radius, agents)
    
    angle = find_steering_angle(robot_vel, new_vel)
    
    prev_pos = curr_pos.copy()
    
    if data.ctrl[0] + angle > 0.4:
        data.ctrl[0] = 0.4
    elif data.ctrl[0] + angle < -0.4:
        data.ctrl[0] = -0.4
    else:
        data.ctrl[0] = data.ctrl[0] + angle
        
    data.ctrl[1] = 2

In [7]:
def RVO(robot_pos, robot_vel, goal_pos, radius, agents):
    
    preferred_velocity = find_goal_direction(goal_pos, robot_pos)
    obstacles = []
    for agent in agents:
        rel_pos = agent[0] - robot_pos
        rel_vel = find_goal_direction(robot_vel, agent[1])
        dist = np.linalg.norm(rel_pos)
        
        if dist < 2*(radius + agent[2]) and dist != 0:
            obstacles.append((rel_pos / dist, rel_vel / dist))
    if not obstacles:
        return preferred_velocity
    else:
        obstacle_velocities = []
        for obstacle in obstacles:
            relative_velocity = obstacle[1]
            dot_product = np.dot(preferred_velocity, relative_velocity)
            if dot_product < 0:
                total_vel_fin = relative_velocity - dot_product * preferred_velocity
                obstacle_velocities.append((relative_velocity + total_vel_fin)/2)
            else:
                t = np.linalg.norm(np.cross(preferred_velocity, obstacle[0])) / np.linalg.norm(relative_velocity)
                total_vel_fin = (1-(3.2*t)) * relative_velocity
                obstacle_velocities.append((relative_velocity + total_vel_fin)/2)
                
        new_velocity = preferred_velocity + sum(obstacle_velocities)
        # print("new_velocity:",new_velocity)
        return new_velocity

In [8]:
trajectory = []
collide = 0
agent_prev_pos = data.site_xpos[1]

while not glfw.window_should_close(window):
    time_prev = data.time

    while (data.time - time_prev < 1.0/60.0):
        
        buddy_pos = data.site_xpos[0]
        human_pos = data.site_xpos[1]
        agent_vel = find_goal_direction(human_pos, prev_pos)
        agents = [[human_pos, agent_vel, 0.25]]
        agent_prev_pos = data.site_xpos[1]
        
        global checkpoints
        if( len(checkpoints)==0):
            collide = 1
            break
            
        move_buddy(buddy_pos, agents, 0.37524)
        
        new_c = Controller(model,data)
        mj.set_mjcb_control(c.controller(model, data, steering = data.ctrl[0], velocity = data.ctrl[1]))
        mj.mj_step(model,data)
        
        if(check_collision() ):
            collide = 1
            break
            
        trajectory.append(np.copy(data.qpos))
        if view == "first":
            cam.lookat[0] = data.site_xpos[1][0]
            cam.lookat[1] = data.site_xpos[1][1]
            cam.lookat[2] = data.site_xpos[1][2] + 0.5
            cam.elevation = 0.0
            cam.distance = 1.0
    
    if collide == 1:
        break
        
    if data.time >= simend:
        break

    # ==================================================================================
    # The below code updates the visualization -- do not modify it!
    # ==================================================================================
    # get framebuffer viewport
    viewport_width, viewport_height = glfw.get_framebuffer_size(window)
    viewport = mj.MjrRect(0, 0, viewport_width, viewport_height)

    # Update scene and render
    mj.mjv_updateScene(model, data, opt, None, cam, mj.mjtCatBit.mjCAT_ALL.value, scene)
    mj.mjr_render(viewport, scene, context)

    # swap OpenGL buffers (blocking call due to v-sync)
    glfw.swap_buffers(window)

    # process pending GUI events, call GLFW callbacks
    glfw.poll_events()
    

glfw.terminate()